To generate counterfactuals, DiCE implements two kinds of methods: model-agnostic and gradient-based. 

* **Model-Agnostic**: These methods apply to any black-box classifier or regressor. They are based on sampling nearby points to an input point, while optimizing a loss function based on proximity (and optionally, sparsity, diversity and feasibility). Use this class of methods for sklearn models. Currently supported methods are:
    * Randomized Search
    * Genetic Search
    * KD Tree Search (for counterfactuals from a given training dataset)
* **Gradient-Based**: These methods apply to differentiable models, such as those returned by deep learning libraries like tensorflow and pytorch. They are based on an explicit loss minimization based on proximity, diversity and feasibility. The method is described in this [paper](https://arxiv.org/abs/1905.07697).

DiCE requires two inputs: a training dataset and a pre-trained ML model. When the training dataset is unknown (e.g., for privacy reasons), it can also work without access to the full dataset

In [58]:
# Sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset  

# DiCE imports
import dice_ml
from dice_ml.utils import helpers  # helper functions

In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
dataset = helpers.load_adult_income_dataset()

This dataset has 8 features. The outcome is income which is binarized to 0 (low-income, <=50K) or 1 (high-income, >50K). 

In [61]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,28,Private,Bachelors,Single,White-Collar,White,Female,60,0
1,30,Self-Employed,Assoc,Married,Professional,White,Male,65,1
2,32,Private,Some-college,Married,White-Collar,White,Male,50,0
3,20,Private,Some-college,Single,Service,White,Female,35,0
4,41,Self-Employed,Some-college,Married,White-Collar,White,Male,50,0


In [62]:
# description of transformed features
adult_info = helpers.get_adult_data_info()
adult_info

{'age': 'age',
 'workclass': 'type of industry (Government, Other/Unknown, Private, Self-Employed)',
 'education': 'education level (Assoc, Bachelors, Doctorate, HS-grad, Masters, Prof-school, School, Some-college)',
 'marital_status': 'marital status (Divorced, Married, Separated, Single, Widowed)',
 'occupation': 'occupation (Blue-Collar, Other/Unknown, Professional, Sales, Service, White-Collar)',
 'race': 'white or other race?',
 'gender': 'male or female?',
 'hours_per_week': 'total work hours per week',
 'income': '0 (<=50K) vs 1 (>50K)'}

Split the dataset into train and test sets.

In [63]:
target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)

Given the train dataset, we construct a data object for DiCE. Since continuous and discrete features have different ways of perturbation, we need to specify the names of the continuous features. DiCE also requires the name of the output variable that the ML model will predict.

In [64]:
# Step 1: dice_ml.Data
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### Loading the ML model

DiCE supports sklearn, tensorflow and pytorch models. 

The variable *backend* below indicates the implementation type of DiCE we want to use. Four backends are supported: sklearn, TensorFlow 1.x with backend='TF1', Tensorflow 2.x with backend='TF2', and PyTorch with backend='PYT'. 

Below we show use a trained classification model using sklearn. 

In [65]:
numerical = ["age", "hours_per_week"]
categorical = x_train.columns.difference(numerical)

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model = clf.fit(x_train, y_train)

## Generating counterfactual examples using DiCE

We now initialize the DiCE explainer, which needs a dataset and a model. DiCE provides local explanation for the model *m* and requires an query input whose outcome needs to be explained. 

In [66]:
# Using sklearn backend
m = dice_ml.Model(model=model, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

The `method` parameter specifies the explanation method. DiCE supports three methods for sklearn models: random sampling, genetic algorithm search, and kd-tree based generation.  

The next code snippet shows how to generate and visualize counterfactuals. The first argument of the `generate_counterfactuals` method is the _query instances_ on which counterfactuals are desired. This can be a dataframe with one or more rows. 

Below we provide a sample input whose outcome is 0 (low-income) as per the ML model object *m*. Given the query input, we can now generate counterfactual explanations to show perturbed inputs from the original input where the ML model outputs class 1 (high-income). The last column shows the output of the classifier: `income-output` >=0.5 is class 1 and `income-output`<0.5 is class 0. 

In [67]:
e1 = exp.generate_counterfactuals(x_test[0:1], total_CFs=10, desired_class="opposite")
e1.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  2.51it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Bachelors,-,-,-,Male,-,1
1,-,Government,-,-,Professional,-,-,-,1
2,-,-,Masters,-,White-Collar,-,-,-,1
3,-,Government,-,Single,-,-,-,-,1
4,-,-,Masters,-,Professional,-,-,-,1
5,-,Self-Employed,School,-,-,-,-,-,1
6,-,-,Doctorate,-,White-Collar,-,-,-,1
7,-,-,Prof-school,-,-,-,-,58,1
8,67,-,Prof-school,-,-,-,-,-,1
9,-,-,Prof-school,-,White-Collar,-,-,-,1


That's it! You can try generating counterfactual explanations for other examples using the same code. 
It is also possible to restrict the features to vary while generating the counterfactuals, and to specify permitted range of features within which the counterfactual should be generated. 

In [68]:
# Changing only age and education
e2 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=10,
                                  desired_class="opposite",
                                  features_to_vary=["education", "occupation"]
                                  )
e2.visualize_as_dataframe(show_only_changes=True)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  2.53it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Masters,-,Sales,-,-,-,1
1,-,-,Bachelors,-,Professional,-,-,-,1
2,-,-,Masters,-,Other/Unknown,-,-,-,1
3,-,-,Prof-school,-,Professional,-,-,-,1
4,-,-,Prof-school,-,-,-,-,-,1
5,-,-,Prof-school,-,White-Collar,-,-,-,1
6,-,-,Assoc,-,Sales,-,-,-,1
7,-,-,Assoc,-,White-Collar,-,-,-,1
8,-,-,Masters,-,White-Collar,-,-,-,1
9,-,-,Doctorate,-,White-Collar,-,-,-,1


In [69]:
# Restricting age to be between [20,30] and Education to be either {'Doctorate', 'Prof-school'}.
e3 = exp.generate_counterfactuals(x_test[0:1],
                                  total_CFs=10,
                                  desired_class="opposite",
                                  permitted_range={'age': [20, 30], 'education': ['Doctorate', 'Prof-school']})
e3.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  2.63it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,24,-,Prof-school,-,-,-,-,-,1
1,-,-,Doctorate,-,-,-,Male,-,1
2,-,-,Prof-school,-,-,-,-,10,1
3,-,-,Prof-school,-,-,-,Male,-,1
4,21,-,Prof-school,-,-,-,-,-,1
5,-,Government,Prof-school,-,-,-,-,-,1
6,-,-,Doctorate,-,Professional,-,-,-,1
7,-,-,Prof-school,-,-,-,-,32,1
8,-,-,Prof-school,-,-,-,-,76,1
9,-,-,Prof-school,-,-,-,-,70,1


## Working with deep learning models (TensorFlow and PyTorch)

We now show examples of gradient-based methods with Tensorflow and Pytorch models. Since the gradient-based methods _optimize_ the loss rather than simply sampling some points, they can be slower to generate counterfactuals. The loss is defined by three component: **validity** (does the CF have the desired model output), **proximity** (distance of CF from original point should be low), and **diversity** (multiple CFs should change different features). 

### Explaining a Tensorflow model

In [81]:
backend = 'TF2'  # needs tensorflow installed
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
# Step 2: dice_ml.Model
m = dice_ml.Model(model_path=ML_modelpath, backend=backend, func="ohe-min-max")

2025-03-13 12:20:15.459752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-13 12:20:15.459779: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


We want to note that the time required to find counterfactuals with Tensorflow 2.x's eager style of execution is significantly greater than that with TensorFlow 1.x's graph execution.

Based on the data object d and the model object m, we can now instantiate the DiCE class for generating explanations.

In [82]:
# Step 3: initiate DiCE
exp = dice_ml.Dice(d, m, method="gradient")

2025-03-13 12:20:44.397768: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-13 12:20:44.397791: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-13 12:20:44.397804: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (adarsh-NBR-WAX9): /proc/driver/nvidia/version does not exist
2025-03-13 12:20:44.397989: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [83]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(x_test[1:2], total_CFs=4, desired_class="opposite")
# visualize the result, highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

Diverse Counterfactuals found! total time taken: 00 min 56 sec
Query instance (original outcome : 0.6710000038146973)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50.0,Other/Unknown,Some-college,Married,Other/Unknown,White,Male,40.0,0.671



Diverse Counterfactual set (new outcome: 0.0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,23.0,-,-,-,-,-,-,13.0,0
1,17.0,-,-,-,-,Other,-,-,0
2,-,-,School,Single,-,-,-,-,0
3,62.0,-,-,Divorced,Blue-Collar,-,-,-,0


### Explaining a Pytorch model

Just change the backend variable to 'PYT' to use DiCE with PyTorch. Below, we use a pre-trained ML model in PyTorch which produces high accuracy comparable to other baselines. For convenience, we include the sample trained model with the DiCE package. Additionally, we need to provide a data transformer function that converts input dataframe into one-hot encoded/numeric format. 

In [70]:
backend = 'PYT'  # needs pytorch installed
ML_modelpath = helpers.get_adult_income_modelpath(backend=backend)
m = dice_ml.Model(model_path=ML_modelpath, backend=backend,  func="ohe-min-max")

Instantiate the DiCE class with the new PyTorch model object *m*. 

In [71]:
exp = dice_ml.Dice(d, m, method="gradient")

In [76]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(x_test[0:1], total_CFs=10, desired_class="opposite")
# highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Bachelors,-,-,-,-,47,1
1,-,-,-,-,White-Collar,-,-,80,1
2,38,-,Doctorate,-,-,-,-,-,1
3,-,-,Doctorate,-,Other/Unknown,-,-,-,1
4,-,-,Masters,-,-,-,-,76,1
5,66,-,Prof-school,-,-,-,-,-,1
6,-,-,Prof-school,-,-,-,-,67,1
7,-,-,Bachelors,-,-,-,Male,-,1
8,36,-,Prof-school,-,-,-,-,-,1
9,59,-,Prof-school,-,-,-,-,-,1


We can also use method-agnostic explainers like "random" or "genetic". 

In [77]:
m = dice_ml.Model(model_path=ML_modelpath, backend=backend, func="ohe-min-max")
exp = dice_ml.Dice(d, m, method="random")

In [80]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(x_test[0:2], total_CFs=10, desired_class="opposite")
# highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 2/2 [00:01<00:00,  1.33it/s]

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,29,Private,HS-grad,Married,Blue-Collar,White,Female,38,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,Doctorate,-,Service,-,-,-,1
1,-,-,Prof-school,-,-,-,Male,-,1
2,-,-,Prof-school,-,Service,-,-,-,1
3,-,-,Masters,-,-,-,Male,-,1
4,67,-,-,-,Professional,-,-,-,1
5,-,-,Masters,-,-,-,-,78,1
6,40,-,-,-,-,-,-,93,1
7,70,-,-,-,-,-,-,86,1
8,56,-,Masters,-,-,-,-,-,1
9,43,-,Masters,-,-,-,-,-,1


Query instance (original outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,50,Other/Unknown,Some-college,Married,Other/Unknown,White,Male,40,1



Diverse Counterfactual set (new outcome: 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,-,-,-,Separated,-,-,-,60,0
1,21,-,Masters,-,-,-,-,-,0
2,24,-,-,-,-,-,-,-,0
3,35,-,-,-,White-Collar,-,-,-,0
4,-,-,-,Widowed,-,Other,-,-,0
5,-,-,-,-,White-Collar,-,-,7,0
6,-,-,-,-,-,-,Female,-,0
7,20,-,Masters,-,-,-,-,-,0
8,-,-,Doctorate,Single,-,-,-,-,0
9,-,-,-,Widowed,-,-,-,52,0
